Nhiệm vụ 4 – Merge dữ liệu

In [ ]:
#Đọc dự liệu
import pandas as pd
citizen_df = pd.read_csv("../data_clean/citizen_info_clean.csv")
feedback_df = pd.read_csv("../data_clean/feedback_log_clean.csv")
resolution_df = pd.read_csv("../data_clean/resolution_time_clean.csv")


In [ ]:
# 4.1Merge citizen_info và feedback_log theo citizen_id để gắn thêm thông tin khu vực cho từng phản ánh. 
feedback_citizen = feedback_df.merge(
    citizen_df[['citizen_id', 'area']],
    on='citizen_id',
    how='left'
)

feedback_citizen.head()


,feedback_id,citizen_id,category,channel,department,submit_time,area
0,FB1000,CD113,môi trường,truc tiep,phong tnmt,2024-01-21 12:47:00,quan 1
1,FB1001,CD114,ha tang,web site,phong tnmt,NaN,NaN
2,FB1002,CD142,ha tang,hotline,ubnd phuong,2024-01-30 15:36:00,NaN
3,FB1003,cd102,môi trường,hot line,cong an,NaN,phuong a
4,FB1004,CD105,moi truong,hotline,ubnd phuong,2024-01-20 17:07:00,phuong a


In [ ]:
# 4.2Merge thêm resolution_time theo feedback_id để tính thời gian xử lý. 
full_feedback = feedback_citizen.merge(
    resolution_df,
    on='feedback_id',
    how='left'
)

full_feedback.head()


,feedback_id,citizen_id,category,channel,department,submit_time,area,resolved_time,status,resolution_hours
0,FB1000,CD113,môi trường,truc tiep,phong tnmt,2024-01-21 12:47:00,quan 1,2024-02-02 05:50:00,từ chối,NaN
1,FB1000,CD113,môi trường,truc tiep,phong tnmt,2024-01-21 12:47:00,quan 1,NaN,da xu ly,24.0
2,FB1001,CD114,ha tang,web site,phong tnmt,NaN,NaN,NaN,NaN,NaN
3,FB1002,CD142,ha tang,hotline,ubnd phuong,2024-01-30 15:36:00,NaN,NaN,NaN,NaN
4,FB1003,cd102,môi trường,hot line,cong an,NaN,phuong a,NaN,NaN,NaN


In [ ]:
#4.3 Phát hiện các bản ghi feedback_log không có bản ghi tương ứng trong resolution_time (chưa có kết quả xử lý). 
unresolved_feedback = full_feedback[
    full_feedback['resolved_time'].isna()
]
unresolved_feedback.head()



,feedback_id,citizen_id,category,channel,department,submit_time,area,resolved_time,status,resolution_hours
1,FB1000,CD113,môi trường,truc tiep,phong tnmt,2024-01-21 12:47:00,quan 1,NaN,da xu ly,24.0
2,FB1001,CD114,ha tang,web site,phong tnmt,NaN,NaN,NaN,NaN,NaN
3,FB1002,CD142,ha tang,hotline,ubnd phuong,2024-01-30 15:36:00,NaN,NaN,NaN,NaN
4,FB1003,cd102,môi trường,hot line,cong an,NaN,phuong a,NaN,NaN,NaN
8,FB1005,CD126,ha tang,app,cong an,NaN,NaN,NaN,đã xử lý,8.0


In [18]:
#số lượng phản ánh chưa xử lý
unresolved_feedback.head()
unresolved_count = unresolved_feedback.shape[0]
unresolved_count

106

In [ ]:
#4.4 Phát hiện các feedback_id trong resolution_time không tồn tại trong feedback_log (dữ liệu sai lệch). 
invalid_resolution = resolution_df[    #feedback_id tồn tại trong resolution_time nhưng không có trong feedback_log
    ~resolution_df['feedback_id'].isin(feedback_df['feedback_id'])
]

invalid_resolution.head()


,feedback_id,resolved_time,status,resolution_hours


In [ ]:
#Số lượng bản ghi sai lệch
invalid_count = invalid_resolution.shape[0]
invalid_count


0

In [23]:
#Xuất file
full_feedback.to_csv(
    "full_feedback_merged.csv",
    index=False
)
